In [29]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Теперь-реализуем-собственный-алгоритм." data-toc-modified-id="Теперь-реализуем-собственный-алгоритм.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Теперь реализуем собственный алгоритм.</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп».
Задача - разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [30]:
data = pd.read_csv('insurance.csv')
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные хорошо подготовлены, все признаки соответствуют их типу и нет пропусков.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

 Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

**Ответ:**
Нет, не изменится, так как формула предсказания по итогу не изменилась, а значит вектор весов w а также w0 не изменились

**Обоснование:**

Признаки умножают а обратимую матрицу. Наши признаки - матрица X, P - матрица на которую умножают.
Получается надо проверить, изменяться ли веса если наша матрица будет не X а XP. Вектор наших весов - w. Поэтому будем работать с нашей формулой обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Подставим вместо X ,XP и попытаемся преобразовать:
$$
w = ((XP)^T (XP))^{-1} (XP)^T y
$$

По свойству транспонирования матриц:

$$
(XP)^T = P^TX^T
$$


$$
w = (P^TX^T (XP))^{-1} P^TX^T y
$$

по свойству
$$
(X^T)^{-1} = (X)^{-1}^T
$$

$$
w = (P^T)^{-1}(X)^{-1}^T (X)^{-1}(P)^{-1} P^TX^T y
$$

$$
w = (P^T)^{-1}(X^TX)^{-1}(P)^{-1} P^TX^T y
$$
Преобразовали все что могли, теперь, подставим w в нашу формулу предсказания, так как прежде всего нам надо проверить, ухудшаться ли наши предсказания. Наша оригинальная формула:

$$
a = Xw
$$
Подставим вместо X наше значение:
$$
a = XPw
$$
Теперь подставим в нашу формулу преобразованное значение w:
$$
a = XP(P^T)^{-1} (X^TX)^{-1}P^{-1}P^TX^Ty
$$
$$
a = XPP^{-1}(P^T)^{-1}P^T (X^TX)^{-1}X^Ty
$$
Вспомним свойство обратной матрицы $$AA^-1=E$$
А также, что любая матрица, умноженная на единичную это сама матрица. Исходя из это , можно сократить нашу запись выше
$$
a = X(X^TX)^{-1}X^Ty
$$
В итоге мы получили ту же формулу для предсказания, если бы мы не умножани нашу матрицу X на P. Как итог - дествительно ничего не поменялось.


## Алгоритм преобразования

Воспользуемся тем алгоритмом, который мы доказали выше. То есть умножим наши признаки на нашу обратимую матрицу, которые мы создадим.

Подбробнее распишем наш алгоритм:
1. Создадим рандомную обратимую матрицу
2. Умножим наши features на нашу обратимую матрицу, тем самым зашифровав данные пользователей
3. Реализуем деление на тестовую и тренировочную выборку
4. Обучим нашу модель линейной регрессии на наших данных

## Реализация алгоритма и проверка

### 1. Обучим на наших данных без шифорвания и посчитаем метрику r2

In [32]:
features =data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']

features_train,features_test,target_train,target_test = train_test_split(features,target,test_size=0.2,random_state=42)

In [33]:
model = LinearRegression()
model.fit(features_train,target_train)
y_pred = model.predict(features_test)
print(r2_score(target_test,y_pred))

0.43686949231380134


### 2. Обучим на наших данных c шифорванием и посчитаем метрику r2

Создадим квадратную матрицу с рандомными значениями и проверим ее обратимость

In [34]:
code_matrix = np.random.random((features.shape[1],features.shape[1]))
print(code_matrix)

[[0.79775537 0.12263791 0.5537098  0.08611623]
 [0.29545532 0.26863993 0.67595627 0.15847503]
 [0.38226768 0.5690297  0.95750061 0.77853464]
 [0.23706142 0.8301808  0.91790766 0.55144267]]


In [35]:
code_matrix@np.linalg.inv(code_matrix)

array([[ 1.00000000e+00, -2.22044605e-16,  1.73472348e-18,
         5.55111512e-17],
       [ 0.00000000e+00,  1.00000000e+00, -5.55111512e-17,
         0.00000000e+00],
       [ 1.11022302e-16,  4.44089210e-16,  1.00000000e+00,
         0.00000000e+00],
       [ 1.11022302e-16,  4.44089210e-16, -5.55111512e-17,
         1.00000000e+00]])

Получили единичную матрицу, значит наша матрица обратима

Теперь обучим нашу модель с зашифрованными данными

In [36]:
code_features = features@code_matrix

features_train,features_test,target_train,target_test = train_test_split(code_features,target,test_size=0.2,random_state=42)

In [37]:
lin_model = LinearRegression()
lin_model.fit(features_train,target_train)
y_pred = lin_model.predict(features_test)
print(r2_score(target_test,y_pred))

0.4368694923161953


Сравнив две метрики на зашифрованных и незашифрованных данных, можно сделать вывод, что наша метрика не изменилась, а значит, шифрование корректно

## Вывод

В данном проекте была проведена работа, по шифрованию данных. Для начала были изучены данные, которые были хорошо подготовлены и у них не оказалось пропусков. Далее был доказан алгоритм шифрования, суть которого, умножить матрицу признаков на обратимую матрицу, тем самым зашифровать данные. Доказав, что предсказание не изменится, так как вектор весов не поменяет свое значение также, как и формула предсказания не поменяет свой вид. Далее, докасзанный алгоритм был проанализирован и проверен. Сначала была обучена модель без шифрования, а потом с выполнением алгоритма шифрования. В каждом из способов была посчитана метрика r2. Две полученные метрики оказались одинаковыми(обе оказались равными 0.4368694923161953), что еще раз доказывает алгоритм на практике.